#### Step 1: Import libraties

In [38]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, MetaData, Table

#### Step 2: Load the Excel file

In [39]:
# Define the path to the excel file
file_path = 'Nashville_Housing_Data.xlsx'

# Load the excel file into a pandas ExcelFile object
excel_file = pd.ExcelFile(file_path)

# Get a list of all sheet names in the excel file
sheet_names = excel_file.sheet_names

# Print the list of sheet names
print(sheet_names)

['property_sales']


#### Step 3: Store each sheet in the dictionary

In [40]:
# Create an empty dictionary to store the data from each sheet 
housing_data = {} # Dictionary for storing the data

# Loop through each sheet name in the list of sheet names
for sheet in sheet_names:
    # Read the data from each sheet and store it in the dictionary with sheet names as keys
    housing_data[sheet] = excel_file.parse(sheet) # Use 'parse' nethod instead of 'pd.read_excel'

# Now the data from each sheet is stored in the housing_data dictionary

# Get all keys (sheet names) from the housing_data dictionary
housing_data.keys()

dict_keys(['property_sales'])

#### Step 4: Process 'property_sales' DataFrame

In [41]:
housing_data['property_sales'].head(3)

,UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
0,2045,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",2013-04-09,240000,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE","1808 FOX CHASE DR, GOODLETTSVILLE, TN",2.3,GENERAL SERVICES DISTRICT,50000.0,168200.0,235700.0,1986.0,3.0,3.0,0.0
1,16918,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",2014-06-10,366000,20140619-0053768,No,"BONER, CHARLES & LESLIE","1832 FOX CHASE DR, GOODLETTSVILLE, TN",3.5,GENERAL SERVICES DISTRICT,50000.0,264100.0,319000.0,1998.0,3.0,3.0,2.0
2,54582,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",2016-09-26,435000,20160927-0101718,No,"WILSON, JAMES E. & JOANNE","1864 FOX CHASE DR, GOODLETTSVILLE, TN",2.9,GENERAL SERVICES DISTRICT,50000.0,216200.0,298000.0,1987.0,4.0,3.0,0.0


In [42]:
housing_data['property_sales'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56477 entries, 0 to 56476
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   UniqueID         56477 non-null  int64         
 1   ParcelID         56477 non-null  object        
 2   LandUse          56477 non-null  object        
 3   PropertyAddress  56448 non-null  object        
 4   SaleDate         56477 non-null  datetime64[ns]
 5   SalePrice        56477 non-null  int64         
 6   LegalReference   56477 non-null  object        
 7   SoldAsVacant     56477 non-null  object        
 8   OwnerName        25261 non-null  object        
 9   OwnerAddress     26015 non-null  object        
 10  Acreage          26015 non-null  float64       
 11  TaxDistrict      26015 non-null  object        
 12  LandValue        26015 non-null  float64       
 13  BuildingValue    26015 non-null  float64       
 14  TotalValue       26015 non-null  float

In [43]:
# Iterate over each sheet name and its corresponding data in the housing_data dictionary
for sheet_name, data in housing_data.items():
    # Remove leading and trailing whitespaces from all column names in the current sheet's data
    data.columns = data.columns.str.strip()

#### Step 5: Create the database Nashville_housing_db

In [44]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host = 'localhost',  # Host where the MySQL server is running
    user = 'root',       # MySQL username
    password = '123456'  # MySQL password
)

cursor = conn.cursor()  # Create a cursor object to interact with the database

# Create the database if it doesn't already exist
cursor.execute('CREATE DATABASE IF NOT EXISTS Nashville_housing_db')
conn.commit()  # Commit the transaction to save changes

# Close the cursor and the connection
cursor.close()
conn.close()

#### Step 6: Load data from the dictionary into MySQL tables

In [45]:
# Define connection parameters
host = "localhost"  # Host where the MySQL server is running
database = 'nashville_housing_db'  # Name of the database to connect to
user = 'root'  # MySQL username
password = '123456'  # MySQL password

# Create an SQLAlchemy engine to connect to MySQL using pymysql as the DBAPI
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# Load DataFrames from the dictionary into corresponding MySQL tables
for sheet_name, data in housing_data.items():  # Loop through each sheet name and DataFrame
    data.to_sql(sheet_name, con=engine, if_exists='replace', index=False)  # Insert data into MySQL, replace table if it exists